In [ ]:
# ------------------------------------------------------------------------
#
# TITLE - jeans_df_tests.ipynb
# AUTHOR - James Lane
# PROJECT - tng-dfs
#
# ------------------------------------------------------------------------
#
# Docstring:
'''Test the application of the Jeans equation to mock data from DFs
'''

__author__ = "James Lane"

In [ ]:
### Imports

## Basic
import numpy as np
import sys, os, pdb, copy
import h5py
import dill as pickle
from astropy import units as apu

## Matplotlib
import matplotlib
from matplotlib import pyplot as plt

## Galpy
from galpy import orbit, potential, df

## Scipy
import scipy.interpolate

sys.path.insert(0,'../../src/')
from tng_dfs import util as putil
from tng_dfs import tree as ptree
from tng_dfs import cutout as pcutout

### Notebook setup
%matplotlib inline
plt.style.use('../../src/mpl/project.mplstyle') # This must be exactly here
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2

In [ ]:
# Keywords
cdict = putil.load_config_to_dict()
keywords = ['DATA_DIR','RO','VO','ZO','LITTLE_H']
data_dir,ro,vo,zo,h = putil.parse_config_dict(cdict,keywords)

# Figure directory
fig_dir = './fig/jeans_df_tests/'

### First prepare some DFs and draw samples

In [ ]:
# hpot = potential.HernquistPotential(amp=1.,a=10*apu.kpc,ro=ro,vo=vo)
# hdf = df.isotropicHernquistdf(hpot,ro=ro,vo=vo)
# orbs = hdf.sample(n=100000,return_orbit=True)

# fig = plt.figure()
# ax = fig.add_subplot(111)

# ax.hist(orbs.r().value,bins=50,range=(0,50))
# rs = np.linspace(1,50,1000)*apu.kpc
# ax.plot(rs.value,potential.evaluateDensities(hpot,rs,0).value*rs.value**2*270000)

# ax.set_xlim(0,50)

# fig.show()

### Now calculate the Jeans quantity

The Jeans equation
$\frac{\mathrm{d} (\nu\,\overline{v^2_r})}{\mathrm{d} r} +\,\nu\,
\left(\frac{\mathrm{d} \Phi}{\mathrm{d} r}+
\frac{2\overline{v_r^2}-\overline{v_\theta^2}-\overline{v_\phi^2}}{r}\right)= 0$

This equation has units of 

$J \equiv [\ell]^{-4}[v]^{2}$

And so one way of normalizing if not using real units is to divide by 

$\mathrm{ro}^{-4}\mathrm{vo}^{2}$

In [ ]:
def calculate_spherical_jeans_quantities(orbs,pot,r_range=[0,100],n_bin=10,
    norm_by_galpy_scale_units=False,calculate_pe_with_pot=False,ro=ro,vo=vo):
    '''calculate_spherical_jeans_quantities:
    
    Calculate the quantities used in the spherical Jeans equation.
    
    Args:
        orbs (Orbits) - Orbits object containing particles / kinematic sample
        pot (Potential) - Potential object representing the gravitational 
            potential experienced by orbs
        r_range (optional, list) - Range of radii to consider, in kpc 
            [default: [0,100]]
        n_bin (optional, int) - Number of bins to use in calculating Jeans
            equation, note derivative quantities will be calculated with 
            n_bin+1 bins [default: 10]
        norm_by_galpy_scale_units (optional, bool) - If True, normalize the
            Jeans equation by galpy scale units [default: False]
        calculate_pe_with_pot (optional, bool) - If True, calculate the 
            potential at the bin centers, rather than the mean potential of the 
            orbs in the bin [default: False]
        ro (optional, float) - Distance scale in kpc [default: 8.275]
        vo (optional, float) - Velocity scale in km/s [default: 220.]
    
    Returns:
        qs (tuple) - Tuple of kinematic quantities used to calculate Jeans
            equation, output from calculate_spherical_jeans_quantities, 
            in order: dnuvr2dr,dphidr,nu,vr2,vp2,vt2,rs
    '''
    orbs = copy.deepcopy(orbs)
    orbs.turn_physical_on(ro=ro,vo=vo)
    pot = copy.deepcopy(pot)
    pot.turn_physical_on(ro=ro,vo=vo)

    ## Determine bins for kinematic properties
    
    # First need bins for derivatives, one more bin than for the data itself, 
    # since we're taking derivatives
    n_dr_bin = n_bin+1
    dr_bin_edge = np.linspace(r_range[0],r_range[1],n_dr_bin+1)
    dr_bin_cents = (dr_bin_edge[1:]+dr_bin_edge[:-1])/2
    # dr_bin_delta = dr_bin_edge[1:]-dr_bin_edge[:-1]

    # One fewer bin for data, since we're taking derivatives. The edges are 
    # the derivative bin centers
    bin_edge = copy.deepcopy(dr_bin_cents)
    bin_cents = (bin_edge[1:]+bin_edge[:-1])/2
    # bin_delta = bin_edge[1:]-bin_edge[:-1]

    # Bin the data, derivative quantities first
    nuvr2 = np.zeros_like(dr_bin_cents)
    phi = np.zeros_like(dr_bin_cents)
    # Non-derivative quantities
    nu = np.zeros_like(bin_cents)
    vr2 = np.zeros_like(bin_cents)
    vt2 = np.zeros_like(bin_cents)
    vp2 = np.zeros_like(bin_cents)

    rs = orbs.r(use_physical=True).to(apu.kpc).value
    pe = potential.evaluatePotentials(pot,orbs.R(),orbs.z(),
        use_physical=True).to(apu.km**2/apu.s**2).value
    pe_bin_cents = potential.evaluatePotentials(pot,dr_bin_cents*apu.kpc,
        0*apu.kpc,use_physical=True).to(apu.km**2/apu.s**2).value

    # Derivative quantities
    for i in range(len(dr_bin_cents)):
        bin_mask = (rs>=dr_bin_edge[i]) & (rs<dr_bin_edge[i+1])
        n_in_bin = np.sum( bin_mask )
        bin_vol = 4*np.pi/3*(dr_bin_edge[i+1]**3-dr_bin_edge[i]**3)
        dr_nu = n_in_bin/bin_vol
        dr_vr2 = np.mean(orbs.vr(use_physical=True).to(apu.km/apu.s).value
            [bin_mask]**2.)
        if calculate_pe_with_pot:
            phi[i] = pe_bin_cents[i]
        else:
            phi[i] = np.mean(pe[bin_mask])
        nuvr2[i] = dr_nu*dr_vr2
    dphidr = np.diff(phi)/np.diff(dr_bin_cents)
    dnuvr2dr = np.diff(nuvr2)/np.diff(dr_bin_cents)

    # Non-derivative quantities
    for i in range(len(bin_cents)):
        bin_mask = (rs>=bin_edge[i]) & (rs<bin_edge[i+1])
        n_in_bin = np.sum( bin_mask )
        bin_vol = 4*np.pi/3*(bin_edge[i+1]**3-bin_edge[i]**3)
        nu[i] = n_in_bin/bin_vol
        vr2[i] = np.mean(orbs.vr(use_physical=True).to(apu.km/apu.s).value
            [bin_mask]**2.)
        vp2[i] = np.mean(orbs.vtheta(use_physical=True).to(apu.km/apu.s).value
            [bin_mask]**2.)
        vt2[i] = np.mean(orbs.vT(use_physical=True).to(apu.km/apu.s).value
            [bin_mask]**2.)
    
    # Normalize densities by number of orbits so they're proper number 
    # densities
    nu /= len(orbs)
    dnuvr2dr /= len(orbs)

    if norm_by_galpy_scale_units:
        nu = nu*(ro**3)
        vr2 = vr2/(vo**2)
        vp2 = vp2/(vo**2)
        vt2 = vt2/(vo**2)
        bin_cents = bin_cents/ro
        dphidr = dphidr*ro/(vo**2)
        dnuvr2dr = dnuvr2dr*(ro**4)/(vo**2)

    return dnuvr2dr,dphidr,nu,vr2,vp2,vt2,bin_cents

def calculate_spherical_jeans(orbs,pot,r_range=[0,100],n_bin=10,
    norm_by_galpy_scale_units=False,norm_by_nuvr2_r=True,
    calculate_pe_with_pot=False,return_kinematics=True,ro=ro,vo=vo):
    '''calculate_spherical_jeans:

    Calculate the spherical Jeans equation for a given kinematic sample

    Args:
        orbs (Orbits) - Orbits object containing particles / kinematic sample
        pot (Potential) - Potential object representing the gravitational 
            potential experienced by orbs
        r_range (optional, list) - Range of radii to consider, in kpc 
            [default: [0,100]]
        n_bin (optional, int) - Number of bins to use in calculating Jeans
            equation, note derivative quantities will be calculated with 
            n_bin+1 bins [default: 10]
        norm_by_galpy_scale_units (optional, bool) - If True, normalize the
            Jeans equation by galpy scale units [default: False]
        norm_by_nuvr2_r (optional, bool) - If True, normalize the Jeans equation
            by nu*vr^2/r [default: True]
        calculate_pe_with_pot (optional, bool) - If True, calculate the 
            potential at the bin centers, rather than the mean potential of the 
            orbs in the bin [default: False]
        return_kinematics (optional, bool) - If True, return the kinematics
            used to calculate the Jeans equation [default: True]
        ro (optional, float) - Distance scale in kpc [default: 8.275]
        vo (optional, float) - Velocity scale in km/s [default: 220.]
    
    Returns:
        J (np.ndarray) - Jeans equation, may be normalized
        rs (np.ndarray) - Radii at which Jeans equation is calculated
        qs (tuple) - Tuple of kinematic quantities used to calculate Jeans
            equation, output from calculate_spherical_jeans_quantities, 
            in order: dnuvr2dr,dphidr,nu,vr2,vp2,vt2,rs
    '''
    # Compute the 
    qs = calculate_spherical_jeans_quantities(orbs,pot,r_range=r_range,
        n_bin=n_bin,norm_by_galpy_scale_units=norm_by_galpy_scale_units,
        calculate_pe_with_pot=calculate_pe_with_pot,ro=ro,vo=vo)

    dnuvr2dr,dphidr,nu,vr2,vp2,vt2,rs = qs

    # Compute the Jeans equation
    J = nu*(dphidr + (2*vr2-vp2-vt2)/rs) + dnuvr2dr

    # Normalize by nu*vr^2/r if desired. Note that this returns the same 
    # answer regardless of whether using physical or galpy units.
    if norm_by_nuvr2_r and not norm_by_galpy_scale_units:
        J = J/(nu*vr2/rs)

    if return_kinematics:
        return J,rs,qs
    else:
        return J,rs

### Try a Hernquist profile

In [ ]:
pot = potential.HernquistPotential(amp=1e12*apu.M_sun,a=10*apu.kpc,ro=ro,vo=vo)
hdf = df.isotropicHernquistdf(pot,ro=ro,vo=vo)
orbs = hdf.sample(n=100000,return_orbit=True)

nit = 500
nbins = 10
norm_by_nuvr2_r = True
norm_by_galpy_scale_units = False
Js = np.zeros((nit,nbins))
# rs = np.zeros((nit,nbins))
for i in range(nit):

    orbs = hdf.sample(n=10000,return_orbit=True)
    _J,rs,_qs = calculate_spherical_jeans(orbs,pot,r_range=[0,50],n_bin=nbins,
        norm_by_nuvr2_r=norm_by_nuvr2_r,
        norm_by_galpy_scale_units=norm_by_galpy_scale_units)
    Js[i,:] = _J

fig = plt.figure()
ax = fig.add_subplot(111)

ax.plot(rs, np.median(Js,axis=0), color='Black')
J_upper = np.percentile(Js, 84, axis=0)
J_lower = np.percentile(Js, 16, axis=0)
ax.fill_between(rs, J_lower, J_upper, color='Black', alpha=0.5)
ax.axhline(0, color='Black', linestyle='--', linewidth=0.5)

ax.set_xlim(0,50)
ax.set_xlabel('r [kpc]')
if norm_by_nuvr2_r:
    ax.set_ylabel(r'$J / (\nu \bar{v_{r}^{2}} / r)$')
else:
    ax.set_ylabel('$J$')
ax.set_title('Hernquist, scale 10 kpc')
fig.savefig(fig_dir+'hernquist_j.png', dpi=300)
# fig.show()
plt.close()

### Create a function to plot diagnostics for sampled kinematics

In [ ]:
def plot_jeans_diagnostics(Js,rs,qs,adf,r_range):

    plot_jeans_sigmas = True
    data_color = 'Black'
    data_linewidth = 2.
    truth_color = 'Red'

    pot = adf._pot
    denspot = adf._denspot

    fig = plt.figure(figsize=(12,8))
    gs = fig.add_gridspec(nrows=4,ncols=3)
    axs = np.array([fig.add_subplot(gs[:2,0]),
                    fig.add_subplot(gs[0,1]),
                    fig.add_subplot(gs[1,1]),
                    fig.add_subplot(gs[:2,2]),
                    fig.add_subplot(gs[2:,0]),
                    fig.add_subplot(gs[2,1]),
                    fig.add_subplot(gs[3,1]),
                    fig.add_subplot(gs[2:,2])
                    ])
    # axs = fig.subplots(nrows=2,ncols=3).flatten()

    # J in the first panel
    lJ,mJ,uJ = np.percentile(Js, [16,50,84], axis=0)
    axs[0].plot(rs, mJ, color=data_color, linewidth=data_linewidth)
    axs[0].fill_between(rs, lJ, uJ, color='Black', alpha=0.25)
    axs[0].axhline(0, color='Black', linestyle='--', linewidth=0.5)
    axs[0].set_xlim(0,50)
    axs[0].set_xlabel('r [kpc]')
    if norm_by_nuvr2_r:
        axs[0].set_ylabel(r'$J / (\nu \bar{v_{r}^{2}} / r)$')
    else:
        axs[0].set_ylabel('$J$')
    fig.suptitle('Hernquist, scale 10 kpc')

    # Density in the second upper panel
    lnu,mnu,unu = np.percentile(qs[2], [16,50,84], axis=0)
    axs[1].plot(rs, mnu, color=data_color, linewidth=data_linewidth)
    denspot_dens = potential.evaluateDensities(denspot,rs*apu.kpc,0)
    denspot_norm = potential.mass(denspot,r_range[1]*apu.kpc)-\
                   potential.mass(denspot,r_range[0]*apu.kpc)
    denspot_dens = (denspot_dens/(denspot_norm)).to(apu.kpc**-3).value
    axs[1].plot(rs, denspot_dens,#*mnu[0]/denspot_dens[0], 
        color=truth_color, linestyle='--')
    axs[1].fill_between(rs, unu, lnu, color='Black', alpha=0.25)
    # axs[1].set_xlim(0,50)
    axs[1].set_xscale('log')
    axs[1].set_yscale('log')
    # axs[1].set_xlabel(r'r [kpc]')
    axs[1].set_ylabel(r'$\nu$')

    # Delta density in the second lower panel
    # dnu = (qs[2] - denspot_dens*mnu[0]/denspot_dens[0])/(denspot_dens*mnu[0]/denspot_dens[0])
    dnu = (qs[2]-denspot_dens)/denspot_dens
    ldnu,mdnu,udnu = np.percentile(dnu, [16,50,84], axis=0)
    axs[2].plot(rs, mdnu, color=data_color, linewidth=data_linewidth)
    axs[2].fill_between(rs, udnu, ldnu, color='Black', alpha=0.25)
    axs[2].axhline(0, color='Black', linestyle='--', linewidth=0.5)
    axs[2].set_xscale('log')
    axs[2].set_xlabel(r'r [kpc]')
    axs[2].set_ylabel(r'$\Delta \nu$ [fractional]')

    # Beta in the third panel
    beta = 1 - (qs[4]+qs[5])/(2*qs[3])
    lbeta,mbeta,ubeta = np.percentile(beta, [16,50,84], axis=0)
    axs[3].plot(rs, mbeta, color=data_color, linewidth=data_linewidth)
    axs[3].fill_between(rs, ubeta, lbeta, color='Black', alpha=0.25)
    axs[3].axhline(0, color='Black', linestyle='--', linewidth=0.5)
    axs[3].set_xlim(0,50)
    axs[3].set_xlabel(r'r [kpc]')
    axs[3].set_ylabel(r'$\beta$')

    # Radial velocity dispersions in the fourth panel, polar and azimuthal 
    # in the fifth upper/lower panels
    colors = ['DodgerBlue','Crimson','DarkOrange']
    v2_names = [r'$\bar{v_{r}^{2}}$',
                r'$\bar{v_{\phi}^{2}}$',
                r'$\bar{v_{\theta}^{2}}$',]
    for i in range(3):
        for j in range(3):
            lv2,mv2,uv2 = np.percentile(qs[j+3], [16,50,84], axis=0)
            if i == j:
                axs[i+4].plot(rs, mv2, color=colors[j], 
                    linewidth=data_linewidth+2, zorder=2)
                axs[i+4].fill_between(rs, uv2, lv2, color=colors[i], alpha=0.25, 
                    zorder=1)
                if plot_jeans_sigmas:
                    mom = [0]*len(rs)
                    for k in range(len(rs)):
                        if i == 0:
                            mom[k] = adf.vmomentdensity(rs[k]*apu.kpc,2,0)
                        elif i in [1,2]:
                            mom[k] = adf.vmomentdensity(rs[k]*apu.kpc,0,2)/2
                        mom[k] /= adf.vmomentdensity(rs[k]*apu.kpc,0,0)
                        mom[k] = mom[k].to_value(apu.km**2/apu.s**2)
                    mom = np.asarray(mom)
                    axs[i+4].plot(rs, mom, color='Black', alpha=1.,
                        linestyle='--', linewidth=1., zorder=3)
            else:
                axs[i+4].plot(rs, mv2, color=colors[j], alpha=1., 
                    linestyle='--', linewidth=1., zorder=3)
        axs[i+4].set_xlim(0,50)
        if i in [0,2]:
            axs[i+4].set_xlabel(r'r [kpc]')
        axs[i+4].set_ylabel(v2_names[i])
        axs[i+4].set_yscale('log')
    
    # dphi/dr in the sixth panel
    ldphidr,mdphidr,udphidr = np.percentile(qs[1], [16,50,84], axis=0)
    axs[7].plot(rs, mdphidr, color=data_color, linewidth=data_linewidth)
    axs[7].fill_between(rs, udphidr, ldphidr, color='Black', alpha=0.25)
    negpf = -potential.evaluaterforces(pot,rs*apu.kpc,0).\
        to(apu.km**2/apu.s**2/apu.kpc).value
    axs[7].plot(rs, negpf, color=truth_color, linestyle='--')
    axs[7].set_xlim(0,50)
    axs[7].set_xlabel(r'r [kpc]')
    axs[7].set_ylabel(r'$\mathrm{d}\Phi/\mathrm{d}r$')
    axs[7].set_yscale('log')
    
    return fig,axs

### Eddington DF Hernquist

In [ ]:
# Plot the same but with more information
nit = 50
nbins = 10
nsamp = int(1e4)
bin_r_range = [1,50] # The range of radii to use for the bins
samp_r_range = [bin_r_range[0]/2,bin_r_range[1]*4] # Sampling range
norm_by_nuvr2_r = True
norm_by_galpy_scale_units = False
Js = np.zeros((nit,nbins))
qs = np.zeros((7,nit,nbins)) # dnuvr2dr,dphidr,nu,vr2,vp2,vt2,rs

# mwhalo = potential.NFWPotential(conc=15,mvir=1,ro=ro,vo=vo)
# mwhalo = potential.HernquistPotential(amp=1e12*apu.M_sun,a=10*apu.kpc,ro=ro,vo=vo)
pot = potential.HernquistPotential(amp=1.,a=10*apu.kpc,ro=ro,vo=vo)
# hdf = df.isotropicHernquistdf(hpot,ro=ro,vo=vo)
edf = df.eddingtondf(pot=pot,ro=ro,vo=vo,rmax=samp_r_range[1]*apu.kpc)
# hdf = df.constantbetadf(pot=mwhalo,denspot=hpot,beta=0.,ro=ro,vo=vo)

# rs = np.zeros((nit,nbins))
for i in range(nit):
    print('iteration = {0}/{1}'.format(i+1,nit),end='\r')
    orbs = edf.sample(n=nsamp,return_orbit=True,rmin=samp_r_range[0]*apu.kpc)
    _J,rs,_qs = calculate_spherical_jeans(orbs,pot,r_range=bin_r_range,
        n_bin=nbins,norm_by_nuvr2_r=norm_by_nuvr2_r,
        norm_by_galpy_scale_units=norm_by_galpy_scale_units)
    Js[i,:] = _J
    for j in range(len(_qs)):
        qs[j,i,:] = _qs[j]

In [ ]:
fig,axs = plot_jeans_diagnostics(Js,rs,qs,edf,samp_r_range)
fig.suptitle('Hernquist, scale 10 kpc')
fig.tight_layout()
fig.savefig(fig_dir+'hernquist.png',dpi=300)
# fig.show()
plt.close()

### Constant beta Hernquists

In [ ]:
betas = [-0.5,0.,0.3,0.5,0.9]

# Plot the same but with more information
nit = 50
nbins = 10
nsamp = int(1e4)
bin_r_range = [0,50] # The range of radii to use for the bins
samp_r_range = [0,1e4] # Sampling range
norm_by_nuvr2_r = True
norm_by_galpy_scale_units = False


for i in range(len(betas)):
    Js = np.zeros((nit,nbins))
    qs = np.zeros((7,nit,nbins)) # dnuvr2dr,dphidr,nu,vr2,vp2,vt2,rs

    # mwhalo = potential.NFWPotential(conc=15,mvir=1,ro=ro,vo=vo)
    # mwhalo = potential.HernquistPotential(amp=1e12*apu.M_sun,a=10*apu.kpc,ro=ro,vo=vo)
    pot = potential.HernquistPotential(amp=1.,a=10*apu.kpc,ro=ro,vo=vo)
    # hdf = df.isotropicHernquistdf(hpot,ro=ro,vo=vo)
    cdf = df.constantbetaHernquistdf(pot=pot,beta=betas[i],ro=ro,vo=vo)
    # hdf = df.constantbetadf(pot=mwhalo,denspot=hpot,beta=0.,ro=ro,vo=vo)

    # rs = np.zeros((nit,nbins))
    for j in range(nit):
        print('iteration = {0}/{1}'.format(j+1,nit),end='\r')
        orbs = cdf.sample(n=nsamp,return_orbit=True,rmin=samp_r_range[0]*apu.kpc)
        _J,rs,_qs = calculate_spherical_jeans(orbs,pot,r_range=bin_r_range,
            n_bin=nbins,norm_by_nuvr2_r=norm_by_nuvr2_r,
            norm_by_galpy_scale_units=norm_by_galpy_scale_units)
        Js[j,:] = _J
        for k in range(len(_qs)):
            qs[k,j,:] = _qs[k]
    
    fig,axs = plot_jeans_diagnostics(Js,rs,qs,cdf,samp_r_range)
    axs[3].axhline(betas[i],color='Black',linestyle='--')
    fig.suptitle('Hernquist scale 10 kpc, beta = {0}'.format(betas[i]))
    fig.tight_layout()
    fig.savefig(fig_dir+'hernquist_beta_{0}.png'.format(betas[i]),dpi=300)
    # fig.show()
    plt.close()

### Eddington DF Hernquist embedded in MW NFW Halo

In [ ]:
nit = 50
nbins = 10
nsamp = int(1e4)
bin_r_range = [1,50] # The range of radii to use for the bins
samp_r_range = [bin_r_range[0]/2,bin_r_range[1]*4] # Sampling range
norm_by_nuvr2_r = True
norm_by_galpy_scale_units = False
Js = np.zeros((nit,nbins))
qs = np.zeros((7,nit,nbins)) # dnuvr2dr,dphidr,nu,vr2,vp2,vt2,rs

pot = potential.NFWPotential(conc=15,mvir=1,ro=ro,vo=vo)
# mwhalo = potential.HernquistPotential(amp=1e12*apu.M_sun,a=10*apu.kpc,ro=ro,vo=vo)
denspot = potential.HernquistPotential(amp=1.,a=10*apu.kpc,ro=ro,vo=vo)
# hdf = df.isotropicHernquistdf(hpot,ro=ro,vo=vo)
edf = df.eddingtondf(pot=pot,denspot=denspot,ro=ro,vo=vo,
                     rmax=samp_r_range[1]*apu.kpc)
# hdf = df.constantbetadf(pot=mwhalo,denspot=hpot,beta=0.,ro=ro,vo=vo)

# rs = np.zeros((nit,nbins))
for i in range(nit):
    print('iteration = {0}/{1}'.format(i+1,nit),end='\r')
    orbs = edf.sample(n=nsamp,return_orbit=True,rmin=samp_r_range[0]*apu.kpc)
    _J,rs,_qs = calculate_spherical_jeans(orbs,pot,r_range=bin_r_range,
        n_bin=nbins,norm_by_nuvr2_r=norm_by_nuvr2_r,
        norm_by_galpy_scale_units=norm_by_galpy_scale_units)
    Js[i,:] = _J
    for j in range(len(_qs)):
        qs[j,i,:] = _qs[j]

In [ ]:
# # Random scratch work
# df.jeans.sigmar(hdf._pot,10.*apu.kpc,hdf._denspot,beta=0.)
# hdf.vmomentdensity(rs[0],2,0)/hdf.vmomentdensity(rs[0],0,0)

# print(hdf.vmomentdensity(rs[0],2,0))
# print(hdf.sigmar(rs[0])**2)
# print(potential.mass(hpot,1e10,ro=ro,vo=vo))
# print(hdf.vmomentdensity(rs[0],2,0)/(potential.evaluateDensities(hdf._denspot,rs[0],0)/potential.mass(hpot,1e10,ro=ro,vo=vo)).to(apu.kpc**-3))

# denspot_dens = potential.evaluateDensities(hpot,rs*apu.kpc,0)
# (denspot_dens/(potential.mass(hpot,1e10,ro=ro,vo=vo))).to(apu.kpc**-3)
# mnu/1e4

In [ ]:
fig,axs = plot_jeans_diagnostics(Js,rs,qs,edf,samp_r_range)
fig.suptitle('Hernquist scale 10 kpc, embedded in NFW halo')
fig.tight_layout()
fig.savefig(fig_dir+'hernquist_embedded_nfw.png',dpi=300)
# fig.show()
plt.close()

In [ ]:
# Embedded constant beta profiles not working yet, probably need to be 
# pre-computed

# betas = [-0.5,0.,0.3,0.5,0.9]

# # Plot the same but with more information
# nit = 50
# nbins = 10
# nsamp = int(1e4)
# bin_r_range = [0,50] # The range of radii to use for the bins
# samp_r_range = [0,1e4] # Sampling range
# norm_by_nuvr2_r = True
# norm_by_galpy_scale_units = False


# for i in range(len(betas)):
#     Js = np.zeros((nit,nbins))
#     qs = np.zeros((7,nit,nbins)) # dnuvr2dr,dphidr,nu,vr2,vp2,vt2,rs

#     pot = potential.NFWPotential(conc=15,mvir=1,ro=ro,vo=vo)
#     # mwhalo = potential.HernquistPotential(amp=1e12*apu.M_sun,a=10*apu.kpc,ro=ro,vo=vo)
#     denspot = potential.HernquistPotential(amp=1.,a=10*apu.kpc,ro=ro,vo=vo)
#     # hdf = df.isotropicHernquistdf(hpot,ro=ro,vo=vo)
#     cdf = df.constantbetadf(pot=pot,denspot=denspot,beta=betas[i],ro=ro,vo=vo)
#     # hdf = df.constantbetadf(pot=mwhalo,denspot=hpot,beta=0.,ro=ro,vo=vo)

#     # rs = np.zeros((nit,nbins))
#     for j in range(nit):
#         print('iteration = {0}/{1}'.format(j+1,nit),end='\r')
#         orbs = cdf.sample(n=nsamp,return_orbit=True,rmin=samp_r_range[0]*apu.kpc)
#         _J,rs,_qs = calculate_spherical_jeans(orbs,pot,r_range=bin_r_range,
#             n_bin=nbins,norm_by_nuvr2_r=norm_by_nuvr2_r,
#             norm_by_galpy_scale_units=norm_by_galpy_scale_units)
#         Js[j,:] = _J
#         for k in range(len(_qs)):
#             qs[k,j,:] = _qs[k]
    
#     fig,axs = plot_jeans_diagnostics(Js,rs,qs,cdf,samp_r_range)
#     axs[3].axhline(betas[i],color='Black',linestyle='--')
#     fig.suptitle('Hernquist scale 10 kpc, beta = {0}'.format(betas[i]))
#     fig.tight_layout()
#     fig.show()

### Eddington DF, multiple potentials embedded in MW Halo NFW
- Hernquist
- NFW
- Truncated power law
- Power laws with $\alpha \in [1.5,2.5,3.5,4.5]$

In [ ]:
pot = potential.NFWPotential(conc=15,mvir=1,ro=ro,vo=vo)
denspots = [
    potential.NFWPotential(a=20*apu.kpc,ro=ro,vo=vo),
    potential.PowerSphericalPotentialwCutoff(alpha=3.5,rc=30*apu.kpc,ro=ro,vo=vo),
    potential.PowerSphericalPotential(alpha=1.5,ro=ro,vo=vo),
    potential.PowerSphericalPotential(alpha=2.5,ro=ro,vo=vo),
    potential.PowerSphericalPotential(alpha=3.5,ro=ro,vo=vo),
    potential.PowerSphericalPotential(alpha=4.5,ro=ro,vo=vo),
]
denspot_names = [
    'NFW scale 20 kpc',
    'Exponential cutoff (alpha=3.5, rc=30 kpc)',
    'alpha=1.5',
    'alpha=2.5',
    'alpha=3.5',
    'alpha=4.5',
]
fig_names = [
    'nfw',
    'exp_cutoff',
    'alpha_1.5',
    'alpha_2.5',
    'alpha_3.5',
    'alpha_4.5',
]

nit = 50
nbins = 10
nsamp = int(1e4)
bin_r_range = [1,50] # The range of radii to use for the bins
samp_r_range = [bin_r_range[0]/2,bin_r_range[1]*2] # Sampling range
norm_by_nuvr2_r = True
norm_by_galpy_scale_units = False
Js = np.zeros((nit,nbins))
qs = np.zeros((7,nit,nbins)) # dnuvr2dr,dphidr,nu,vr2,vp2,vt2,rs


for i in range(len(denspots)):

    edf = df.eddingtondf(pot=pot,denspot=denspots[i],ro=ro,vo=vo,
        rmax=samp_r_range[1]*apu.kpc)
    Js = np.zeros((nit,nbins))
    qs = np.zeros((7,nit,nbins)) # dnuvr2dr,dphidr,nu,vr2,vp2,vt2,rs

    # rs = np.zeros((nit,nbins))
    for j in range(nit):
        print('iteration = {0}/{1}'.format(j+1,nit),end='\r')
        orbs = edf.sample(n=nsamp,return_orbit=True,rmin=samp_r_range[0]/ro)
        _J,rs,_qs = calculate_spherical_jeans(orbs,pot,r_range=bin_r_range,
            n_bin=nbins, norm_by_nuvr2_r=norm_by_nuvr2_r,
            norm_by_galpy_scale_units=norm_by_galpy_scale_units)
        Js[j,:] = _J
        for k in range(len(_qs)):
            qs[k,j,:] = _qs[k]
    
    fig,axs = plot_jeans_diagnostics(Js,rs,qs,edf,r_range=samp_r_range)
    fig.suptitle(denspot_names[i]+', embedded in NFW MW halo')
    fig.tight_layout()
    fig.savefig(fig_dir+'{0}_embedded_nfw.png'.format(fig_names[i]))
    # fig.show()
    plt.close()